In [9]:
from itertools import product

DEBUG = True

In [10]:

def get_binary_states(nodes, node_values, exclude_values_list):
    # Check which dictionaries in exclude_values_list agree with node_values
    valid_exclude_values = []
    
    for exclude_values in exclude_values_list:
        valid = True
        for node, value in node_values.items():
            if node not in exclude_values or value != exclude_values[node]:
                valid = False
                break

        if valid:
            valid_exclude_values.append(exclude_values)

    # Generate all binary states that agree with node_values
    return generate_states(nodes, node_values, valid_exclude_values)

def generate_states(nodes, node_values, valid_exclude_values=None):
    # List of nodes to generate binary values for (nodes that aren't in node_values or exclude_values)
    nodes_to_generate = [node for node in nodes if node not in node_values]
    
    # Get all binary combinations (0 or 1) for the remaining nodes
    binary_combinations = product('01', repeat=len(nodes_to_generate))
    
    # Generate the binary states
    binary_states = []
    for combination in binary_combinations:
        # Create a copy of node_values and update it with the combination
        state = node_values.copy()
        
        # Map the combination to the nodes we need to generate values for
        for i, node in enumerate(nodes_to_generate):
            state[node] = int(combination[i])
        
    
        # Check if the current state agrees with valid exclude values
        valid_state = True
        for exclude_values in valid_exclude_values:
            valid_state = False
            for node, value in exclude_values.items():
                if state[node] != value:
                    valid_state = True
                    break
            if not valid_state:
                break
        
        if valid_state:
            # Convert the state dictionary to a binary string
            binary_state = ''.join(str(state[node]) for node in nodes)
            binary_states.append(binary_state)
    
    return binary_states

# Example usage
nodes = ['A', 'B', 'C', 'D']
node_values = {'A': 1}
exclude_values_list = [{'A':1, 'B': 1, 'C':1}, {'A':1, 'C': 0}, {'A':0}, {'D': 0}]

binary_states = get_binary_states(nodes, node_values, exclude_values_list)
print(binary_states)

['1010', '1011']


In [11]:
nodes = ['A', 'B', 'C', 'D']
SD_nodes = [{}, {'A':1, 'D':0}, {'B':0}, {'A':1, 'B':0, 'D':0}, {'A':0, 'B':0}, {'A':0, 'B':0, 'C':1, 'D':1}]

for SD_node in SD_nodes:
    print(SD_node)
    print(get_binary_states(nodes, SD_node, [node for node in SD_nodes if node != SD_node]))

{}
['0100', '0101', '0110', '0111', '1101', '1111']
{'A': 1, 'D': 0}
['1100', '1110']
{'B': 0}
['1001', '1011']
{'A': 1, 'B': 0, 'D': 0}
['1000', '1010']
{'A': 0, 'B': 0}
['0000', '0001', '0010']
{'A': 0, 'B': 0, 'C': 1, 'D': 1}
['0011']


In [12]:
def get_SD_node_states(nodes, SD_nodes):
    
    """
    When given a list of node names and a list of SD nodes,
    returns a list of states that correspond to each SD node.
    """

    SD_node_states = []    
    for SD_node in SD_nodes:
        other_SD_nodes = [node for node in SD_nodes if node != SD_node]
        states = get_binary_states(nodes, SD_node, other_SD_nodes)

        SD_node_states.append(states)
    
    return SD_node_states


nodes = ['A', 'B', 'C', 'D']
SD_nodes = [{}, {'A':1, 'D':0}, {'B':0}, {'A':1, 'B':0, 'D':0}, {'A':0, 'B':0}, {'A':0, 'B':0, 'C':1, 'D':1}]

SD_node_states = get_SD_node_states(nodes, SD_nodes)

print(SD_node_states)

[['0100', '0101', '0110', '0111', '1101', '1111'], ['1100', '1110'], ['1001', '1011'], ['1000', '1010'], ['0000', '0001', '0010'], ['0011']]


In [13]:

def states_to_indexes(state_groups):

    DEBUG = globals().get('DEBUG', False)
    if DEBUG:
        # Check if all state strings are of the same length
        lengths = [len(state) for state_group in state_groups for state in state_group]
        if len(set(lengths)) != 1:
            raise ValueError("Not all states have the same length")
        
        # Check if all gropus are mutually exclusive
        all_states = [state for state_group in state_groups for state in state_group]
        if len(all_states) != len(set(all_states)):
            raise ValueError("States are not mutually exclusive (duplicates found)")

    # Convert binary strings to integers
    converted = [[int(state, 2) for state in state_group] for state_group in state_groups]
    
    # Sort each sublist
    sorted_states = [sorted(sublist) for sublist in converted]
    
    return sorted_states

# Example usage
states = [['110', '001', '010'], ['111', '000']]
converted_and_sorted_states = states_to_indexes(states)
print(converted_and_sorted_states)

[[1, 2, 6], [0, 7]]
